# Appendix 1 | Ring skymap assumptions

These are the assumptions upon which we base our implementation:

1. All tracts have exactly 4 vertices
1. Except the poles, all tracts are rectangular (to the nearest arcsecond)
1. Within a ring, all tracts share the same upper and lower declination
1. Within a ring, all tracts have the same width, or interval between their RA boundaries
1. Each ring begins calculating their tracts at the same RA

## Set up skymap

We will compare both the inner polygon and outer polygon skymaps. For convenience, we will use the 
pre-calculated tracts from our Full Vertex skymaps.

However, it will be useful to reference the LSST skymap for metadata such as ring numbers, so we'll 
load that, too.

In [10]:
import yaml
from pathlib import Path


package_root = Path.home() / "skymap-to-poly-coords"

In [11]:
inner_fv_path = package_root / "converted_skymaps" / "full_vertex" / "inner_polygons.yaml"

with open(inner_fv_path, "r") as f:
    inner_fv_data = yaml.safe_load(f)

inner_tracts = sorted(inner_fv_data["tracts"].items(), key=lambda x: int(x[0]))
print(f"{len(inner_tracts)} inner tracts loaded.")

18938 inner tracts loaded.


In [12]:
outer_fv_path = package_root / "converted_skymaps" / "full_vertex" / "outer_polygons.yaml"

with open(outer_fv_path, "r") as f:
    outer_fv_data = yaml.safe_load(f)

outer_tracts = sorted(outer_fv_data["tracts"].items(), key=lambda x: int(x[0]))
print(f"{len(outer_tracts)} outer tracts loaded.")

18938 outer tracts loaded.


In [2]:
from skymap_to_poly_coords import load_pickle_skymap

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## 1. All tracts have exactly 4 vertices

In [13]:
def check_all_tracts_have_four_vertices(tract_items):
    for tract_id, tract_data in tract_items:
        polygon = tract_data.get("polygon")
        if len(polygon) != 4:
            raise ValueError(f"⚠️ Tract {tract_id} has {len(polygon)} vertices (expected 4).")
        
    print(f"✅ All {len(tract_items)} tracts have 4 vertices.")

In [ ]:
print("Checking inner tracts...")
check_all_tracts_have_four_vertices(inner_tracts)
print("Checking outer tracts...")
check_all_tracts_have_four_vertices(outer_tracts)

Checking inner tracts...
✅ All 18938 tracts have 4 vertices.
Checking outer tracts...
✅ All 18938 tracts have 4 vertices.


## 2. Except the poles, all tracts are rectangular

In [20]:
def check_all_tracts_are_rectangles(tract_items):
    # Skipping the first and last tracts (poles)
    for tract_id, tract_data in tract_items[1:-1]:  
        polygon = tract_data.get("polygon")

        # Ensure polygon is not None.
        if not polygon:
            raise ValueError(f"⚠️ Tract {tract_id} is missing a polygon.")

        # Round to avoid floating point imprecision (still beyond arcsecond precision)
        ras = [round(point[0], 10) for point in polygon]
        unique_ras = set(ras)

        if len(unique_ras) != 2:
            raise ValueError(f"⚠️ Tract {tract_id} has {len(unique_ras)} unique RA values: {unique_ras}")

    print(f"✅ All {len(tract_items) - 2} tracts (excluding poles) have 2 unique RA values.")

In [21]:
print("Checking all inner tracts are rectangular...")
check_all_tracts_are_rectangles(inner_tracts)
print("Checking all outer tracts are rectangular...")
check_all_tracts_are_rectangles(outer_tracts)

Checking all inner tracts are rectangular...
✅ All 18936 tracts (excluding poles) have 2 unique RA values.
Checking all outer tracts are rectangular...


ValueError: ⚠️ Tract 1 has 4 unique RA values: {19.7533906773, 51.8567458466, 308.1413987525, 340.2453942971}

## 3. Within a ring, all tracts share the same upper and lower declination

In [54]:
def get_dec_bounds(polygon, tract_id, round_amt):
    decs = [round(point[1], round_amt) for point in polygon]

    # Check that there are exactly 2 unique declination values
    unique_decs = set(decs)
    if len(unique_decs) != 2:
        raise ValueError(f"⚠️ Tract {tract_id} has {len(unique_decs)} unique Dec values: {unique_decs}")
    
    # Return min and max.
    return min(decs), max(decs)

In [52]:
from skymap_to_poly_coords import IterateTractAndRing

def check_shared_declination_bounds_within_ring(tract_items, round_amt=10):
    iterate_tract_and_ring = IterateTractAndRing(lsst_skymap._ringNums, add_poles=False)
    ring_dec_bounds = {}
    for tract_id, ring_id in iterate_tract_and_ring:
        # Get declination bounds for the tract.
        polygon = tract_items[tract_id][1]["polygon"]
        min_dec, max_dec = get_dec_bounds(polygon, tract_id, round_amt)

        # Compare with existing bounds for the ring.
        if ring_id in ring_dec_bounds:
            if ring_dec_bounds[ring_id] != (min_dec, max_dec):
                raise ValueError(f"⚠️ Ring {ring_id} has inconsistent declination bounds: "
                                 f"{ring_dec_bounds[ring_id]} vs {min_dec, max_dec}")
        else:
            ring_dec_bounds[ring_id] = (min_dec, max_dec)

    print("✅ All rings have consistent declination bounds across their tracts.")


In [ ]:
print("Checking shared declination bounds within rings...")
check_shared_declination_bounds_within_ring(inner_tracts)
print("Checking shared declination bounds within outer rings (rounding=2)...")
check_shared_declination_bounds_within_ring(outer_tracts,  round_amt=2)
print("Checking shared declination bounds within outer rings (rounding=4)...")
check_shared_declination_bounds_within_ring(outer_tracts,  round_amt=4)

Checking shared declination bounds within rings...
✅ All rings have consistent declination bounds across their tracts.
Checking shared declination bounds within outer rings (rounding=2)...
✅ All rings have consistent declination bounds across their tracts.
Checking shared declination bounds within outer rings (rounding=4)...


ValueError: ⚠️ Tract 1 has 3 unique Dec values: {-88.9406, -87.5342, -87.5341}

## 4. Within a ring, all tracts have the same RA interval

In [ ]:
# Helper functions.

def unwrap_ra_sequence(ras):
    """Unwrap RA values to make them monotonic, accounting for 0/360 crossover."""
    unwrapped = [ras[0]]
    for ra in ras[1:]:
        prev = unwrapped[-1]
        if ra < prev - 180:
            ra += 360
        elif ra > prev + 180:
            ra -= 360
        unwrapped.append(ra)
    return unwrapped

def get_ra_interval(tract_id, tract_data):
    """Get the RA interval of a polygon, handling unwrapped RAs."""
    # Get the polygon for the tract.
    polygon = tract_data.get("polygon")
    if polygon is None:
        return -1.0

    # Get the RAs.
    ras = [round(point[0], 10) for point in polygon]
    unwrapped_ras = unwrap_ra_sequence(ras)
    unique_ras = set(unwrapped_ras)

    # Edge cases and errors.
    if len(unique_ras) < 2:
        return -1.0
    elif len(unique_ras) > 2:
        raise ValueError(f"Polygon has {len(unique_ras)} unique RA values: {unique_ras}")
    
    # Return the interval between the two unique RAs.
    sorted_ras = sorted(unique_ras)
    return (sorted_ras[1] - sorted_ras[0])

In [66]:
import math
from skymap_to_poly_coords import IterateTractAndRing

def check_ra_intervals(tract_items):
    iterate_tract_and_ring = IterateTractAndRing(lsst_skymap._ringNums, add_poles=False)

    ra_intervals = {}
    for tract_id, ring_id in iterate_tract_and_ring:
        tract_data = tract_items[tract_id][1]
        ra_interval = get_ra_interval(tract_id, tract_data)

        if ring_id in ra_intervals:
            if not math.isclose(ra_interval, ra_intervals[ring_id], abs_tol=1e-8):
                raise ValueError(f"⚠️ Ring {ring_id} has inconsistent RA intervals: "
                                 f"{ra_intervals[ring_id]} vs {ra_interval}")
        else:
            ra_intervals[ring_id] = ra_interval
    print("✅ All rings have consistent RA intervals across their tracts.")

In [68]:
print("Checking inner tracts for rectangle shape...")
check_ra_intervals(inner_tracts)
print("Checking outer tracts for rectangle shape...")
check_ra_intervals(outer_tracts)

Checking inner tracts for rectangle shape...
✅ All rings have consistent RA intervals across their tracts.
Checking outer tracts for rectangle shape...


ValueError: Polygon has 4 unique RA values: {379.7533906773, 411.8567458466, 308.1413987525, 340.2453942971}

## 5. Each ring begins calculating their tracts at the same RA

Fun nuance here is that raStart is actually for the center of the first tract in the ring, so you'll 
have to subtract half the ring's width to get to where it actually "starts."

In [70]:
lsst_skymap.config.raStart  # Nice.

0.0

## And finally, two assumptions we make for the reconstruction of the skymap:

### A. CW or CCW

In [71]:
import math

def unwrap_ra_sequence(ras):
    """Unwrap RA values to make them monotonic, accounting for 0/360 crossover."""
    unwrapped = [ras[0]]
    for ra in ras[1:]:
        prev = unwrapped[-1]
        if ra < prev - 180:
            ra += 360
        elif ra > prev + 180:
            ra -= 360
        unwrapped.append(ra)
    return unwrapped

def polygon_orientation_ra_dec(points):
    """Return 'CW' or 'CCW' for a polygon defined by RA/Dec points.

    Parameters
    ----------
    points : list of (float, float)
        List of (RA, Dec) pairs in degrees. Polygon is assumed to be closed
        (first and last point the same) or will be treated as such.

    Returns
    -------
    str
        "CW" for clockwise, "CCW" for counter-clockwise, "Err" if not enough points.
    """
    if len(points) < 3:
        return "ERR"

    ras, decs = zip(*points)
    ras_unwrapped = unwrap_ra_sequence(ras)
    unwrapped_points = list(zip(ras_unwrapped, decs))

    # Shoelace formula: sum over (x_i * y_{i+1} - x_{i+1} * y_i)
    area = 0.0
    n = len(unwrapped_points)
    for i in range(n):
        x0, y0 = unwrapped_points[i]
        x1, y1 = unwrapped_points[(i + 1) % n]
        area += (x0 * y1 - x1 * y0)

    orientation = "CCW" if area > 0 else "CW"
    return orientation


In [84]:
from collections import Counter

def check_winding_order(tract_items):
    ordering_counter = Counter()
    clockwise_tracts = []

    for tract_id, tract_data in tract_items:
        polygon = tract_data.get("polygon")

        order = polygon_orientation_ra_dec(polygon)
        ordering_counter[order] += 1

        if order == "CW":  # We'll record these to verify that they're the poles.
            clockwise_tracts.append(int(tract_id))

    for k, v in ordering_counter.items():
        print(f"- {k}: {v} tracts")
    print(f"{len(clockwise_tracts)} tracts are clockwise: {', '.join(map(str, clockwise_tracts))}\n")

In [85]:
print("Checking inner tracts winding order...")
check_winding_order(inner_tracts)
print("Checking outer tracts winding order...")
check_winding_order(outer_tracts)

Checking inner tracts winding order...
- CW: 2 tracts
- CCW: 18936 tracts
2 tracts are clockwise: 0, 18937

Checking outer tracts winding order...
- CW: 2 tracts
- CCW: 18936 tracts
2 tracts are clockwise: 0, 18937



### B. Start from a consistent corner

In [86]:
def classify_starting_corner(polygon):
    """Classify the starting corner of a polygon based on unwrapped RA/Dec.

    Parameters
    ----------
    polygon : list of (float, float)
        List of (RA, Dec) pairs in degrees. Assumed to be in polygon vertex order.

    Returns
    -------
    str
        One of: "top-left", "top-right", "bottom-left", "bottom-right"
    """
    if len(polygon) < 3:
        raise ValueError("Need at least 3 vertices to classify corners")

    # Unwrap RAs to make comparisons safe
    ras, decs = zip(*polygon)
    unwrapped_ras = unwrap_ra_sequence(ras)
    start_ra, start_dec = unwrapped_ras[0], decs[0]

    min_ra, max_ra = min(unwrapped_ras), max(unwrapped_ras)
    min_dec, max_dec = min(decs), max(decs)

    horiz = "left" if abs(start_ra - min_ra) < 1e-6 else "right"
    vert = "top" if abs(start_dec - max_dec) < 1e-6 else "bottom"

    return f"{vert}-{horiz}"

In [102]:
from collections import Counter

total_tracts = sum(lsst_skymap._ringNums) + 2

def check_starting_corner(tract_items):
    """Check the starting corner classification for all tracts.
    Counts how many tracts fall into each starting corner category.
    """
    starting_corner_counter = Counter()

    for tract_id, tract_data in tract_items:
        polygon = tract_data.get("polygon")

        corner = classify_starting_corner(polygon)
        starting_corner_counter[corner] += 1

    print("Starting corner classification:")
    for corner, count in starting_corner_counter.items():
        print(f"  {corner}: {count} tracts")

In [103]:
print("Checking inner tracts starting corners...")
check_starting_corner(inner_tracts)
print("Checking outer tracts starting corners...")
check_starting_corner(outer_tracts)

Checking inner tracts starting corners...
Starting corner classification:
  bottom-left: 18938 tracts
Checking outer tracts starting corners...
Starting corner classification:
  bottom-right: 9469 tracts
  bottom-left: 9469 tracts
